In [3]:
import pandas as pd
from qwen_omni_utils import process_mm_info
import os

def path_exists_or_nan(x):
    return pd.isna(x) or os.path.exists("/home/is/dwipraseetyo-a/NAS_HAI/Datasets/grpo_3modalities_datasets/" + x)

df = pd.read_csv("/home/is/dwipraseetyo-a/NAS_HAI/Datasets/grpo_3modalities_datasets/validation.csv")
print(df.shape)

df_filtered = df[
    df['audio_file_name'].apply(path_exists_or_nan) |
    df['image_file_name'].apply(path_exists_or_nan)
]
print(df_filtered.shape)
#df = df[df['audio_file_name'].apply(lambda x: os.path.exists("/home/is/dwipraseetyo-a/NAS_HAI/Datasets/grpo_3modalities_datasets/" + x))]
#print(df.shape)

(15070, 5)
(15070, 5)


In [ ]:
/home/is/dwipraseetyo-a/NAS_HAI/Datasets/grpo_3modalities_datasets/validation/l_CODA_TB_0015_93.wav

In [2]:
df

,messages,audio_file_name,solution,identifier,image_file_name
0,"[{'role': 'system', 'content': [{'type': 'text...",validation/l_CODA_TB_0015_93.wav,"<think>Okay, let's see. Based on the provided ...",CODA_TB_0015,NaN
1,"[{'role': 'system', 'content': [{'type': 'text...",validation/l_CODA_TB_0015_94.wav,"<think>Okay, let's see. Based on the provided ...",CODA_TB_0015,NaN
2,"[{'role': 'system', 'content': [{'type': 'text...",validation/l_CODA_TB_0015_180.wav,"<think>Okay, let's see. Based on the provided ...",CODA_TB_0015,NaN
3,"[{'role': 'system', 'content': [{'type': 'text...",validation/l_CODA_TB_0015_125.wav,"<think>Okay, let's see. Based on the provided ...",CODA_TB_0015,NaN
4,"[{'role': 'system', 'content': [{'type': 'text...",validation/l_CODA_TB_0015_262.wav,"<think>Okay, let's see. Based on the provided ...",CODA_TB_0015,NaN
...,...,...,...,...,...
15065,"[{'role': 'system', 'content': [{'type': 'text...",NaN,"<think>Okay, let's see. The chest X-ray shows ...",2PLNNRL738L3,validation/2PLNNRL738L3.png
15066,"[{'role': 'system', 'content': [{'type': 'text...",NaN,"<think>Okay, let's see. The lungs are clear bi...",8OAASGV076K6,validation/8OAASGV076K6.png
15067,"[{'role': 'system', 'content': [{'type': 'text...",NaN,"<think>Okay, let's see. The lungs are clear bi...",4ZI1N5DHMT18,validation/4ZI1N5DHMT18.png
15068,"[{'role': 'system', 'content': [{'type': 'text...",NaN,"<think>Okay, let's see. The chest X-ray shows ...",VVFEC2BNL3LZ,validation/VVFEC2BNL3LZ.png


In [ ]:
from datasets import load_dataset

val_ds = load_dataset("arkiven4/grpo_3modalities_datasets", 
                      split='validation', 
                      trust_remote_code=True)

Repo card metadata block was not found. Setting CardData to empty.


EmptyDatasetError: The directory at hf://datasets/arkiven4/grpo_3modalities_datasets@3f60f1aeaaa74cabe936662bf1b9180da1da6fa2/. doesn't contain any data files

In [ ]:
#df_filtered.to_csv("/home/is/dwipraseetyo-a/NAS_HAI/Datasets/grpo_3modalities_datasets/train.csv", index=False)

In [19]:
df_missing = df.loc[~df.index.isin(df_filtered.index)]

In [22]:
df_missing['messages'].values

array(['[{\'role\': \'system\', \'content\': [{\'type\': \'text\', \'text\': \'\\nYou are an advanced medical assistant AI specialized in analyzing and diagnosing clinical conditions, capable of perceiving auditory and visual inputs. \\nYou can interpret and reason over various medical inputs, including auditory inputs, visual inputs, and patient symptoms, individually or in combination, depending on what is provided. \\nYour task is to analyze the given input, explain your reasoning, and give a possible diagnosis. Always respond in the following format:\\n<think>{Placeholder for Analyzed the available data X-ray findings if present, cough audio characteristics if provided, and reported symptoms to determine the most likely disease.}</think>\\n\\n<answer>{Placeholder for Determined disease}</answer>\\n\'}]}, {\'role\': \'user\', \'content\': [{\'type\': \'text\', \'text\': "<audio>The patient symptoms are smoke lweek is No, weight loss is Yes, gender is Male, weight is 58.0, hemoptysis

In [ ]:
for col in ['audio_file_name', 'image_file_name']:
    for path in df[col].dropna():
        if os.path.exists(path):
            continue  # skip if source exists in new location
        dest_path = os.path.join("/new/location", os.path.basename(path))
        if not os.path.exists(dest_path):
            shutil.move(path, dest_path)

In [10]:
df_missing = df[~df['audio_file_name'].apply(os.path.exists)]

In [11]:
df_missing

,messages,audio_file_name,solution,identifier,image_file_name
0,"[{'role': 'system', 'content': [{'type': 'text...",train/l_CODA_TB_0002_2.wav,"<think>Okay, let's see. Based on the patient's...",CODA_TB_0002,train/nan
1,"[{'role': 'system', 'content': [{'type': 'text...",train/l_CODA_TB_0002_1.wav,"<think>Okay, let's see. Based on the patient's...",CODA_TB_0002,train/nan
2,"[{'role': 'system', 'content': [{'type': 'text...",train/l_CODA_TB_0003_25.wav,"<think>Okay, let's see. Based on the provided ...",CODA_TB_0003,train/nan
3,"[{'role': 'system', 'content': [{'type': 'text...",train/l_CODA_TB_0003_14.wav,"<think>Okay, let's see. Based on the provided ...",CODA_TB_0003,train/nan
4,"[{'role': 'system', 'content': [{'type': 'text...",train/l_CODA_TB_0003_9.wav,"<think>Okay, let's see. Based on the provided ...",CODA_TB_0003,train/nan
...,...,...,...,...,...
100186,"[{'role': 'system', 'content': [{'type': 'text...",train/nan,"<think>Okay, let's see. The chest X-ray findin...",1GOVQY01H4M3,train/1GOVQY01H4M3.png
100187,"[{'role': 'system', 'content': [{'type': 'text...",train/nan,"<think>Okay, let's see. The chest X-ray findin...",RZY9JRW4JK4R,train/RZY9JRW4JK4R.png
100188,"[{'role': 'system', 'content': [{'type': 'text...",train/nan,"<think>Okay, let's see. The chest X-ray findin...",BKZ6GYHYEV2Y,train/BKZ6GYHYEV2Y.png
100189,"[{'role': 'system', 'content': [{'type': 'text...",train/nan,"<think>Okay, let's see. The chest X-ray findin...",0BTN61W0Y4LY,train/0BTN61W0Y4LY.png


In [1]:
%cd /home/is/dwipraseetyo-a/NAS_HAI/Project/Qwen2.5-Omni

import pandas as pd
import numpy as np
import soundfile as sf
from pathlib import Path
import os, librosa, random, pickle, pydicom, requests, torch, re
from pydicom.datadict import keyword_for_tag
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
from IPython.display import Markdown, display
from openai import OpenAI
from PIL import Image
from tqdm import tqdm

import commons, const_variable
random.seed(42)

DATA_PATH = "/home/is/dwipraseetyo-a/NAS_HAI/Datasets/cidrz"

# system_prompt = '''
# You are an advanced medical assistant AI specialized in analyzing and diagnosing clinical conditions, capable of perceiving auditory and visual inputs. 
# You can interpret and reason over various medical inputs, including auditory inputs, visual inputs, and patient symptoms, individually or in combination, depending on what is provided. 
# Your task is to analyze the given input, and give a possible diagnosis. 
# '''

# system_prompt = (
#     "A conversation between User and Advanced medical assistant specialized in analyzing and diagnosing clinical conditions. and the Assistant determines whether the case is Positive or Negative. The assistant "
#     "first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning "
#     "process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "
#     "<think> reasoning process here </think><answer> answer here </answer>"
# )

system_prompt = '''
You are an advanced medical assistant AI specialized in analyzing and diagnosing clinical conditions, capable of perceiving auditory and visual inputs. 
You can interpret and reason over various medical inputs, including auditory inputs, visual inputs, and patient symptoms, individually or in combination, depending on what is provided. 
Your task is to analyze the given input, explain your reasoning, and give a possible diagnosis. Answer are enclosed within <answer> </answer> tags, respectively, i.e., <answer> answer here </answer> .
Always respond in the following format:

## ⚠️ Points to Review and Disclaimer
<If no auditory or visual input is provided>

## 🧠 Overview
<answer> <Positive or Negative Diagnosis> </answer>

## 📋 Observations
**Chest X-ray:**
<Your explanation based on the relevant visual input>"

**Symptoms:**
<Your explanation based on the input symptoms>"

**Audio:**
<Your explanation based on the input audio>"
'''

/home/ldap-users-2/dwipraseetyo-a/Project/Qwen2.5-Omni


In [4]:
for split in ['train', 'dev']:
    df = pd.read_csv(f"{DATA_PATH}/metadata_cut_processed.csv.{split}")
    df_llm_symptoms = ( pd.read_csv(f"datas/reasoning/symptoms/gpt-4o-mini_symptoms.csv.{split}").groupby('barcode', group_keys=False).apply(lambda x: x.sample(1), include_groups=True).reset_index(drop=True) ) 
    df_llm_images = ( pd.read_csv(f"datas/reasoning/xray/medgemma_xray_formatted.csv.{split}").groupby('path_file_image', group_keys=False).apply(lambda x: x.sample(1), include_groups=True).reset_index(drop=True) )
    df = pd.merge(df, df_llm_symptoms, on='barcode', how='left')
    df = pd.merge(df, df_llm_images, on='path_file_image', how='left')
    df = df.rename(columns={'coughdur': 'cough_duration', 'ngtsweats': 'night_sweets', 'weightloss': 'weight_loss', 'body_wt': 'body_weight'})

    case_info = df['ground_truth_tb'].value_counts().reset_index()
    max_count = case_info['count'][0]
    minor_count = case_info['count'][1]
    ratio = int(max_count // minor_count)

    #df_temp = pd.DataFrame(columns=['barcode', 'question', 'answer', 'tb_status', 'audio_path', 'image_path'])
    instruct_array = []
    for now_row in tqdm(df.itertuples(), desc=f"Processing {split}", total=len(df)):
        row_dict = now_row._asdict()
        now_audiopath = "/home/is/dwipraseetyo-a/NAS_HAI/Datasets/cidrz/" +  now_row.path_file_audio
        now_imgaepath = "/home/is/dwipraseetyo-a/NAS_HAI/Datasets/cidrz/" +  now_row.path_file_image
        current_augment = ratio if now_row.ground_truth_tb == 1 else 1

        for i in range(current_augment):
            question = ""
            array_df = [None, None]
            last_sentence_question, modalities = commons.unique_modalities_generator(const_variable.prompt_templates, now_row.path_file_audio)
            last_sentence_question += ". "
            #templates = const_variable.positive_templates if now_row.ground_truth_tb == 1 else const_variable.negative_templates
            #answer = random.choice(templates)
            answer = commons.generate_tb_response(modalities, now_row.llm_analyze_symptoms, now_row.llm_analyze_image, positive=(now_row.ground_truth_tb == 1))       
            
            array_df = [None, None]
            modalities_tags = ""
            if "symptoms" in modalities:
                row_dict = now_row._asdict()
                #selected_feats = random.sample(const_variable.columns_soundfeat, k=random.randint(3, len(const_variable.columns_soundfeat)))
                symptom_descriptions = ", ".join(
                    f"{feat.replace('_', ' ')} is {row_dict[feat]}"
                    for feat in const_variable.columns_soundfeat #selected_feats
                    if row_dict.get(feat) != "Unknown"
                )
                if symptom_descriptions:
                    question += f"The patient symptoms are {symptom_descriptions}."

            if "audio" in modalities:
                modalities_tags += "<audio>"
                array_df[0] = now_audiopath

            if "xray" in modalities:
                modalities_tags += "<image>"
                array_df[1] = now_imgaepath
                xray_descriptions = ", ".join(
                    f"{feat.replace('_', ' ')} is {row_dict[feat]}"
                    for feat in const_variable.columns_imagefeat
                    if row_dict.get(feat) != "Unknown"
                )
                if xray_descriptions:
                    question += f" The chest x-ray metadata are {xray_descriptions}."

            question = question.strip()
            question = question.rstrip(",.")
            if not question.endswith("."):
                question += ". "
            
            question = modalities_tags + "" + question
            # temp_sentence_arry = last_sentence_question.split(",")
            # if len(temp_sentence_arry) >= 2:
            #     temp_sentence_arry[-2] += " " + modalities_tags
            # question += ",".join(temp_sentence_arry)
            question += last_sentence_question

            temp_instruct = {"messages": [
                {"role": "system",
                    "content": [
                        {"type": "text", "text": system_prompt}
                    ],
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": question},
                    ],
                },
                {"role": "assistant", "content": [
                    {"type": "text", "text": answer}]},
            ]}
            if array_df[0] != None:
                temp_instruct["messages"][1]['content'].append({"type": "audio", "audio": array_df[0]})
            if array_df[1] != None:
                temp_instruct["messages"][1]['content'].append({"type": "image", "image": array_df[1]})
            instruct_array.append(temp_instruct)            
            #df_temp.loc[len(df_temp)] = [now_row.barcode, question, answer, now_row.ground_truth_tb, now_audiopath, now_imgaepath]
    with open(f"datas/instruct_sft_balance.pkl.{split}", "wb") as f:
        pickle.dump(instruct_array, f)
    #pd.DataFrame(df_temp).to_csv(f'datas/grpo_balance.csv.{split}', index=False)

/tmp/ipykernel_3057582/3122668694.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_llm_symptoms = ( pd.read_csv(f"datas/reasoning/symptoms/gpt-4o-mini_symptoms.csv.{split}").groupby('barcode', group_keys=False).apply(lambda x: x.sample(1), include_groups=True).reset_index(drop=True) )
/tmp/ipykernel_3057582/3122668694.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_llm_images = ( pd.read_csv(f"datas/reas

In [4]:
import re

In [10]:
re.search(r'<answer>(.*?)</answer>', "<answer> Negative Tuberculosis </answer>", flags=re.IGNORECASE).group(1).strip()

'Negative Tuberculosis'

# Publish To HG

In [ ]:
from huggingface_hub import login
from dotenv import load_dotenv
from datasets import Dataset, concatenate_datasets

load_dotenv()
login(token=os.getenv("HG_AUTH_KEY_WRITE"))

commons.pretty_status("📦 Loading Dataset...")

with open('datas/instruct_grpo_balance.pkl.train', 'rb') as f:
    train_instruct = commons.load_image_PIL(pickle.load(f))

with open('datas/instruct_grpo_balance.pkl.dev', 'rb') as f:
    dev_instruct = commons.load_image_PIL(pickle.load(f))
    
train_dataset = Dataset.from_list(commons.grpo_build_datasets(train_instruct, None)) #
val_dataset = Dataset.from_list(commons.grpo_build_datasets(dev_instruct, None))

combined = concatenate_datasets([train_dataset, val_dataset])
combined.push_to_hub("arkiven4/cirdz-instruct")


| 📦 Loading Dataset... |



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 439/439 [00:04<00:00, 99.13it/s]


Uploading the dataset shards:   0%|          | 0/4 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/arkiven4/cirdz-instruct/commit/8538285dc2f17ddce47fb960f49054e6c7d10419', commit_message='Upload dataset', commit_description='', oid='8538285dc2f17ddce47fb960f49054e6c7d10419', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/arkiven4/cirdz-instruct', endpoint='https://huggingface.co', repo_type='dataset', repo_id='arkiven4/cirdz-instruct'), pr_revision=None, pr_num=None)

In [4]:
from datasets import load_dataset

train_dataset = load_dataset("arkiven4/cirdz-instruct")

README.md:   0%|          | 0.00/533 [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/308M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/305M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/305M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/325M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3965 [00:00<?, ? examples/s]

In [10]:
train_dataset['train'][0]['messages']

[{'content': '<image>The patient symptoms are sex is f, cough duration is 1-2wks, cough productive is yes, haemoptysis is no, chestpain is yes, shortbreath is no, fever is no, night sweets is no, weight loss is no, hiv status is neg, tobacco use is never, body weight is 83.0, height is 157.0, bmi is 34.0. The chest x-ray metadata are Modality is DX, DistanceSourceToDetector is 1000.0, FieldOfViewDimensions is [350, 425], ImagerPixelSpacing is [0.15, 0.15], TargetExposureIndex is 876.0, Sensitivity is 5273.0, PhotometricInterpretation is MONOCHROME1, Rows is 2336, Columns is 2836, PixelIntensityRelationshipSign is 1, WindowCenter is 1633.0, WindowWidth is 4095.0, RescaleIntercept is 0.0, RescaleSlope is 1.0. Analyze the x-ray and clinical symptoms, is it Positive Tuberculosis or Negative Tuberculosis?. ',
  'role': 'user'},
 {'content': "<think>From the given symptoms and x-ray, Let me Analyze your regrading your questions.\n\n## ⚠️ Points to Review and Disclaimer\n* No Audio Present\n\

In [24]:
train_dataset['train'].features

{'messages': [{'content': Value(dtype='string', id=None),
   'role': Value(dtype='string', id=None)}],
 'solution': Value(dtype='string', id=None),
 'audios': Sequence(feature=Value(dtype='binary', id=None), length=-1, id=None),
 'images': [{'bytes': Value(dtype='binary', id=None),
   'path': Value(dtype='null', id=None)}]}

In [28]:
val_ds.features

{'messages': [{'content': Value(dtype='string', id=None),
   'role': Value(dtype='string', id=None)}],
 'solution': Value(dtype='string', id=None),
 'audios': Sequence(feature=Value(dtype='binary', id=None), length=-1, id=None),
 'images': [{'bytes': Value(dtype='binary', id=None),
   'path': Value(dtype='null', id=None)}]}

In [26]:
from datasets import load_dataset
import os
os.environ["HF_HOME"] = "/home/is/dwipraseetyo-a/NAS_HAI/.cache"

val_ds = load_dataset("/home/is/dwipraseetyo-a/NAS_HAI/Datasets/grpo_3modalities_datasets/omnigrpo_dataset.py", 
                      split='validation', data_dir="/home/is/dwipraseetyo-a/NAS_HAI/Datasets/grpo_3modalities_datasets", 
                      trust_remote_code=True)

Repo card metadata block was not found. Setting CardData to empty.


------------------------ New Version Processor2---------------------------


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
[{'content': Value(dtype='string', id=None), 'role': Value(dtype='string', id=None)}]

In [ ]:
Sequence(feature={'content': Value(dtype='string', id=None), 'role': Value(dtype='string', id=None)}, length=-1, id=None)

In [27]:
val_ds.features

{'messages': [{'content': Value(dtype='string', id=None),
   'role': Value(dtype='string', id=None)}],
 'solution': Value(dtype='string', id=None),
 'audios': Sequence(feature=Value(dtype='binary', id=None), length=-1, id=None),
 'images': [{'bytes': Value(dtype='binary', id=None),
   'path': Value(dtype='null', id=None)}]}

In [ ]:
df = pd.read_csv(f"{DATA_PATH}/metadata_cut_processed.csv.test")
df_llm_symptoms = ( pd.read_csv(f"datas/reasoning/symptoms/o4-mini_symptoms.csv.test").groupby('barcode', group_keys=False).apply(lambda x: x.sample(1), include_groups=True).reset_index(drop=True) ) 
df_llm_images = ( pd.read_csv(f"datas/reasoning/xray/medgemma_xray_formatted.csv.test").groupby('path_file_image', group_keys=False).apply(lambda x: x.sample(1), include_groups=True).reset_index(drop=True) )
df = pd.merge(df, df_llm_symptoms, on='barcode', how='left')
df = pd.merge(df, df_llm_images, on='path_file_image', how='left')
df = df.rename(columns={'coughdur': 'cough_duration', 'ngtsweats': 'night_sweets', 'weightloss': 'weight_loss', 'body_wt': 'body_weight'})

instruct_array_positive = []
instruct_array_negative = []

for now_row in tqdm(df.itertuples(), desc=f"Processing Datasets", total=len(df)):
    row_dict = now_row._asdict()
    now_audiopath = "/home/is/dwipraseetyo-a/NAS_HAI/Datasets/cidrz/" +  now_row.path_file_audio
    now_imgaepath = "/home/is/dwipraseetyo-a/NAS_HAI/Datasets/cidrz/" +  now_row.path_file_image

    modalities = ["audio", "xray", "symptoms"]
    key = tuple(modalities)
    if key in const_variable.prompt_templates:
        question = random.choice(const_variable.prompt_templates[key]) + ". "
    answer = commons.generate_tb_response(modalities, now_row.llm_analyze_symptoms, now_row.llm_analyze_image, positive=(now_row.ground_truth_tb == 1))

    if now_row.path_file_audio == 'Unknown':
        modalities = [m for m in modalities if m != "audio"]

    array_df = [None, None]
    if "symptoms" in modalities:
        row_dict = now_row._asdict()
        selected_feats = random.sample(const_variable.columns_soundfeat, k=random.randint(3, len(const_variable.columns_soundfeat)))
        symptom_descriptions = ", ".join(
            f"{feat.replace('_', ' ')} is {row_dict[feat]}"
            for feat in selected_feats
            if row_dict.get(feat) != "Unknown"
        )
        if symptom_descriptions:
            question += f" Also, the patient presents with: {symptom_descriptions}."

    if "audio" in modalities:
        array_df[0] = now_audiopath

    if "xray" in modalities:
        array_df[1] = now_imgaepath
        xray_descriptions = ", ".join(
            f"{feat.replace('_', ' ')} is {row_dict[feat]}"
            for feat in const_variable.columns_imagefeat
            if row_dict.get(feat) != "Unknown"
        )
        if xray_descriptions:
            question += f" Additional chest x-ray metadata include: {xray_descriptions}."

    question = question.strip()
    question = question.rstrip(",.")
    if not question.endswith("."):
        question += "."

    temp_instruct = {"messages": [
        {"role": "system",
            "content": [
                {"type": "text", "text": system_prompt}
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": question},
            ],
        },
        {"role": "assistant", "content": [
            {"type": "text", "text": answer}]},
    ]}
    if array_df[0] != None:
        temp_instruct["messages"][1]['content'].append({"type": "audio", "audio": array_df[0]})
    if array_df[1] != None:
        temp_instruct["messages"][1]['content'].append({"type": "image", "image":array_df[1]})

    if now_row.ground_truth_tb == 1:
        instruct_array_positive.append(temp_instruct)
    elif now_row.ground_truth_tb == 0:
        instruct_array_negative.append(temp_instruct)
        
with open(f"datas/positive_instruct.pkl.test", "wb") as f:
    pickle.dump(instruct_array_positive, f)

with open(f"datas/negative_instruct.pkl.test", "wb") as f:
    pickle.dump(instruct_array_negative, f)

In [ ]:
def clear_memory():
    # Delete variables if they exist in the current global scope
    if "inputs" in globals():
        del globals()["inputs"]
    if "model" in globals():
        del globals()["model"]
    if "processor" in globals():
        del globals()["processor"]
    if "trainer" in globals():
        del globals()["trainer"]
    if "peft_model" in globals():
        del globals()["peft_model"]
    if "bnb_config" in globals():
        del globals()["bnb_config"]
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")


clear_memory()